In [236]:
import os
from os.path import abspath
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import os
import pandas as pd
import warnings
import requests
import json
from livyc import livyc


In [216]:
warehouse_location = abspath('spark-warehouse')


In [217]:
data_livy = {
    "livy_server_url": "localhost",
    "port": "8998",
    "jars": ["org.sqlserver:mssql-jdbc-12.2.0.jre11.jar"]
}

In [218]:
params = {"host": "localhost", "port":"1433", "database": "db", "table":"BIGDATA", "user": "sa", "password": "123"}


In [219]:
conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.sql.adaptive.enabled","true")
conf.set('spark.driver.extraClassPath', r"/home/guilherme/pysparkapp/mssql-jdbc-12.2.0.jre11.jar")
conf.set('spark.executor.extraClassPath', r"/home/guilherme/pysparkapp/mssql-jdbc-12.2.0.jre11.jar")
spark = SparkSession.builder\
            .config(conf=conf)\
            .config("spark.sql.warehouse.dir", warehouse_location)\
            .config("spark.sql.catalogImplementation", "hive") \
            .enableHiveSupport() \
            .getOrCreate()
    
spark

23/05/28 23:53:33 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [220]:
import json, pprint, requests, textwrap

{u'state': u'starting', u'id': 0, u'kind': u'spark'}

{'state': 'starting', 'id': 0, 'kind': 'spark'}

In [221]:
host = 'http://localhost:8998'
data = {'code': 'sc.parallelize(1 to 10).count()'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()


'Unrecognized field "code" (class org.apache.livy.server.interactive.CreateInteractiveRequest), not marked as ignorable (15 known properties: "executorCores", "conf", "driverMemory", "name", "driverCores", "pyFiles", "archives", "kind", "files", "jars", "proxyUser", "numExecutors", "heartbeatTimeoutInSecond", "queue", "executorMemory"])\n at [Source: (org.eclipse.jetty.server.HttpInputOverHTTP); line: 1, column: 11] (through reference chain: org.apache.livy.server.interactive.CreateInteractiveRequest["code"])'

In [222]:
import requests

url = "http://localhost:8998/sessions"

payload = {}
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"from":0,"total":0,"sessions":[]}


In [223]:

host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)

r.json()
{u'state': u'starting', u'id': 0, u'kind': u'spark'}

{'state': 'starting', 'id': 0, 'kind': 'spark'}

In [224]:


url = "http://localhost:8998"

payload = json.dumps({
  "file": "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibs/o2ov2.coding.category-1.0.0-GET.jar",
  "className": "com.nielseniq.o2ov2.catecoding.O2OV2CategoryCodingSplitApp",
  "jars": [
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2CommonLibs/*",
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibs/CommonInfra-2.10.0-O2OV2.jar",
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibs/o2ov2.common-1.0.0.jar"
  ],
  "name": "DEV_O2OV2_CategoryCoding_GET_20221404",
  "driverMemory": "4g",
  "executorMemory": "4g",
  "numExecutors": 7,
  "executorCores": 4
})
headers = {
  'X-Requested-By': 'admin',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


In [230]:

url = "http://localhost:8998/batches/1"

payload = json.dumps({
  "file": "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/o2ov2.coding.keywords-1.0.0.jar",
  "className": "com.nielseniq.o2ov2.keywords.VUBApp",
  "jars": [
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2CommonLibs/*",
    "hdfs://1localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/o2ov2.common-1.0.0.jar",
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/CommonInfra-2.10.0-O2OV2.jar",
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/DataValidator-2.10.0-O2OV2.jar",
    "hdfs://localhost:8998/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/KeywordsCoding-2.10.0-O2OV2.jar"
  ],
  "name": "DEV_O2OV2_VUBCoding_20221410",
  "driverMemory": "4g",
  "executorMemory": "4g",
  "numExecutors": 7,
  "executorCores": 4
})
headers = {
  'X-Requested-By': 'admin',
  'Content-Type': 'application/json'
}

r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)

print(response.text)

In [231]:
data = {'kind': 'pyspark'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'id': 1, u'state': u'idle'}

{'id': 1, 'state': 'idle'}

In [232]:
statements_url = url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'output': None, u'state': u'running', u'id': 0}


{'output': None, 'state': 'running', 'id': 0}

In [233]:
data = {
  'code': textwrap.dedent("""
    import random
    NUM_SAMPLES = 100000
    def sample(p):
      x, y = random.random(), random.random()
      return 1 if x*x + y*y < 1 else 0

    count = sc.parallelize(xrange(0, NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)
    print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)
    """)
}

r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{u'id': 12,
u'output': {u'data': {u'text/plain': u'Pi is roughly 3.136000'},
            u'execution_count': 12,
            u'status': u'ok'},
u'state': u'running'}

('Unrecognized field "code" (class '
 'org.apache.livy.server.interactive.CreateInteractiveRequest), not marked as '
 'ignorable (15 known properties: "executorCores", "conf", "driverMemory", '
 '"name", "driverCores", "pyFiles", "archives", "kind", "files", "jars", '
 '"proxyUser", "numExecutors", "heartbeatTimeoutInSecond", "queue", '
 '"executorMemory"])\n'
 ' at [Source: (org.eclipse.jetty.server.HttpInputOverHTTP); line: 1, column: '
 '11] (through reference chain: '
 'org.apache.livy.server.interactive.CreateInteractiveRequest["code"])')


{'id': 12,
 'output': {'data': {'text/plain': 'Pi is roughly 3.136000'},
  'execution_count': 12,
  'status': 'ok'},
 'state': 'running'}

In [234]:
pyspark_script = """

    from pyspark.sql.functions import udf, col, explode
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
    from pyspark.sql import Row
    from pyspark.sql import SparkSession


    df = spark.read.format("jdbc") \
        .option("url", "jdbc:postgresql://{host}:{port}/{database}") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", "{table}") \
        .option("user", "{user}") \
        .option("password", "{password}") \
        .load()
        
    n_rows = df.count()

    spark.stop()
"""


r = requests.post(host + '/sessions', data=json.dumps(pyspark_script), headers=headers)
pprint.pprint(r.json())


('Cannot construct instance of '
 '`org.apache.livy.server.interactive.CreateInteractiveRequest` (although at '
 'least one Creator exists): no String-argument constructor/factory method to '
 "deserialize from String value ('\n"
 '\n'
 '    from pyspark.sql.functions import udf, col, explode\n'
 '    from pyspark.sql.types import StructType, StructField, IntegerType, '
 'StringType, ArrayType\n'
 '    from pyspark.sql import Row\n'
 '    from pyspark.sql import SparkSession\n'
 '\n'
 '\n'
 '    df = spark.read.format("jdbc")         .option("url", '
 '"jdbc:postgresql://{host}:{port}/{database}")         .option("driver", '
 '"org.postgresql.Driver")         .option("dbtable", "{table}")         '
 '.option("user", "{user}")         .option("password", "{password}")         '
 '.load()\n'
 '        \n'
 '    n_rows = df.count()\n'
 '\n'
 '    spark.stop()\n'
 "')\n"
 ' at [Source: (org.eclipse.jetty.server.HttpInputOverHTTP); line: 1, column: '
 '1]')


In [235]:
host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)

In [ ]:


url = "http://localhost:8999/batches"

payload = json.dumps({
  "file": "hdfs://10.250.36.25:9000/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/o2ov2.coding.cnn-1.0.0.jar",
  "className": "com.nielseniq.o2ov2.cnncoding.O2OPredictionApp",
  "jars": [
    "hdfs://localhost:8999/projects/O2OV2/APP_PACKAGE/DEV/O2OV2CommonLibs/*",
    "hdfs://localhost:8999/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/o2ov2.common-1.0.0.jar",
    "hdfs://localhost:8999/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/CommonInfra-2.10.0-O2OV2.jar",
    "hdfs://localhost:8999/projects/O2OV2/APP_PACKAGE/DEV/O2OV2BizLibsTemp/CNNCoding-2.10.0-O2OV2.jar"
  ],
  "name": "DEV_O2OV2_CNNCoding_20221410",
  "driverMemory": "4g",
  "executorMemory": "4g",
  "numExecutors": 7,
  "executorCores": 4
})
headers = {
  'X-Requested-By': 'admin',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
'''
    ./bin/spark-submit \
  --class org.apache.spark.examples.SparkPi \
  --master local[8] \
  /home/guilherme/pysparkapp/teste.py \
  100

'''